<a href="https://colab.research.google.com/github/hyeamykim/machine-learning-basics/blob/master/Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
#from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import time
import numpy as np
import random

In [ ]:
# import iris dataset
iris = datasets.load_iris()
iris # check how it looks like

{'DESCR': '.. _iris_dataset:\n\nIris plants dataset\n--------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 150 (50 in each of three classes)\n    :Number of Attributes: 4 numeric, predictive attributes and the class\n    :Attribute Information:\n        - sepal length in cm\n        - sepal width in cm\n        - petal length in cm\n        - petal width in cm\n        - class:\n                - Iris-Setosa\n                - Iris-Versicolour\n                - Iris-Virginica\n                \n    :Summary Statistics:\n\n    ============== ==== ==== ======= ===== ====================\n                    Min  Max   Mean    SD   Class Correlation\n    ============== ==== ==== ======= ===== ====================\n    sepal length:   4.3  7.9   5.84   0.83    0.7826\n    sepal width:    2.0  4.4   3.05   0.43   -0.4194\n    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)\n    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)\n    ===========

In [ ]:
# split into data and target
X = iris.data
y = iris.target

In [ ]:
# split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# K-means clustering algorithm

# function for calculating distance
def calc_distance(X1, X2):
    return (sum((X1 - X2)**2))**0.5

# function for assigning a cluster based on closest centroid
def assign_clusters(centroids, cluster_array):
    clusters = []
    for i in range(cluster_array.shape[0]):
        distances = []
        for centroid in centroids:
            distances.append(calc_distance(centroid, cluster_array[i]))
        cluster = [z for z, val in enumerate(distances) if val==min(distances)]
        clusters.append(cluster[0])
    return clusters

# function for calculating new centroids based on each cluster's mean
def calc_centroids(clusters, cluster_array):
    new_centroids = []
    cluster_df = pd.concat([pd.DataFrame(cluster_array),
                            pd.DataFrame(clusters, 
                                         columns=['cluster'])], 
                           axis=1)
    for c in set(cluster_df['cluster']):
        current_cluster = cluster_df[cluster_df['cluster']\
                                     ==c][cluster_df.columns[:-1]]
        cluster_mean = current_cluster.mean(axis=0)
        new_centroids.append(cluster_mean)
    return new_centroids

In [ ]:
def num_row(data):
    return data.shape[0]
def num_col(data):
    return data.shape[1]

class Cluster:
    def __init__(self, data, centroid,k):
        self.data = data
        self.centroid = centroid
        self.k = k
    
    def calc_distance(self):
        data = self.data
        centroid = self.centroid
        #centroid = centroid.T 
        
        distance = np.empty((num_row(data), k))
        
        for i in range(num_row(data)):
            temp = [np.linalg.norm(data[i] - elem) for elem in centroid]
            distance[i] = temp

        #distance = np.linalg.norm(data - centroid)  # don't use for loop, parallelize  

        return distance
    
    def membership(self):
        distance = self.calc_distance()
        membership = np.zeros((num_row(distance),2))
        for i in range(num_row(distance)):
            membership[i][0] = i # serves as index
            membership[i][1] = np.argmin(distance[i]) # assigns cluster number
        return membership

class New_cluster:
    def __init__(self, data, member,k):
        self.data = data
        self.member = member
        self.k = k
        
    def calc_centroid(self):
        data = self.data
        member= self.member
        new_centroid = np.empty((k,num_col(data)))
        for i in range(k):
            new_centroid[i] = member[member[:,-1]==i].mean()
        return new_centroid

In [ ]:
start_time = time.time()

# number of clusters
k=4

# number of iterations
num_iter = 0

# read data
#data=pd.read_csv("C:/Users/heyon/Desktop/Uni/Summer2019/DDA/Absenteeism_at_work.csv", sep=",")
#target

#X = data
#y = target

# split data into train and test set
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


# initialize
idx = np.random.randint(num_row(X_train), size=k)
centroids = X_train[idx,:]
# centroids = X_train.sample(n=k)
#centroids = random.sample(X_train, k)
#new_array = random.sample(old_array,x)
#centroids = np.array(centroids)
#print(centroids.shape)
#centroids

# data = np.array(data)            

# 
while True:

    cluster = Cluster(X_train, centroids,k)
    # get distance and membership info
    dist = cluster.calc_distance()
    memb = cluster.membership()
    
    #dist = np.asarray(dist).ravel().reshape(num_row(X_train),-1)
    #memb = np.asarray(memb, dtype='i').ravel().reshape(num_row(X_train),-1)
    #data = np.hstack((data, memb)) # add membership info in our data
    #print(data.shape)
    new = New_cluster(X_train, memb, k)
    new_centroids = new.calc_centroid()
    #print(new_centroids.shape)
    #new_centroids = new_centroids[:,:-1]
    #print(new_centroids.shape)

    if np.linalg.norm(centroids - new_centroids) >=1e-3:
        centroids = new_centroids
        num_iter+=1
    else:
        print("It took", time.time() - start_time, "to run")
        print("Number of iterations was", num_iter)
        print(centroids)
        break

It took 0.026942729949951172 to run
Number of iterations was 1
[[21.21875    21.21875    21.21875    21.21875   ]
 [26.48       26.48       26.48       26.48      ]
 [30.83333333 30.83333333 30.83333333 30.83333333]
 [26.88888889 26.88888889 26.88888889 26.88888889]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
